In [0]:
dbutils.widgets.text('p_file_date', '2021-03-21')
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
spark.sql(f"""
          CREATE TABLE IF NOT EXISTS f1_presentation.calculated_race_results
          (
              race_year INT,
              team_name STRING,
              driver_id INT,
              driver_name STRING,
              race_id INT,
              position INT,
              points INT,
              calculated_points INT,
              created_date TIMESTAMP,
              updated_date TIMESTAMP
          )
             USING DELTA
          """)

In [0]:
spark.sql(f"""
            CREATE OR REPLACE TEMP VIEW race_results_updated
            AS
            SELECT
                ra.race_year,
                c.name AS team_name,
                d.driver_id,
                d.name AS driver_name,
                re.race_id AS race_id,
                re.position,
                re.points,
                11 - re.position AS calculated_points
            FROM f1_processed.results re
            JOIN f1_processed.drivers d on re.driver_id = d.driver_id
            JOIN f1_processed.constructors c on re.constructor_id = c.constructor_id
            JOIN f1_processed.races ra on re.race_id = ra.race_id
            WHERE re.position <= 10
            AND re.file_date = '{v_file_date}'
            """
  )

In [0]:
%sql
SELECT * FROM race_results_updated
WHERE race_year = 2021;

In [0]:
spark.sql("""
          MERGE INTO f1_presentation.calculated_race_results tgt
        USING race_results_updated upd
        ON (tgt.race_id = upd.race_id AND tgt.driver_id = upd.driver_id)
        WHEN MATCHED THEN
        UPDATE SET tgt.position = upd.position,
                    tgt.points = upd.points,
                    tgt.calculated_points = upd.calculated_points,
                    tgt.updated_date = current_timestamp
        WHEN NOT MATCHED THEN
        INSERT (race_year, team_name, driver_id, driver_name, race_id, position, points, calculated_points, created_date)
        VALUES (race_year, team_name, driver_id, driver_name, race_id, position, points, calculated_points, current_timestamp);    
          """)

In [0]:
%sql
SELECT COUNT(1) FROM race_results_updated

In [0]:
%sql
SELECT * FROM f1_presentation.calculated_race_results